In [1]:
from bokeh.io import output_notebook, show

from bokeh.plotting import figure

import numpy as np

import pandas as pd

import serial
import time
# setup bokeh to output to the jupyter notebook
output_notebook()


Loading BokehJS ...

In [2]:
# This set's up the serial connection for the ECE544 Project2 Uartlite connection
print("setting up port")
ser = serial.Serial('COM5', 115200, timeout=1)
print(ser)


setting up port
Serial<id=0x1e8f06e5470, open=True>(port='COM5', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False)


In [3]:
def update(df):
    data_str = ser.readline()
    temp = data_str.decode()
    # print(temp)
    
    temp_dict = {}
    for token in temp.split(','):
        if 'ActualRpm' in token:
            values = [x.strip() for x in token.split(':')]
            if (len(values) == 2) and (values[0] == 'ActualRpm'):
                temp_dict.update(dict([values]))
        elif 'DesiredRpm' in token:
            values = [x.strip() for x in token.split(':')]
            if (len(values) == 2) and (values[0] == 'DesiredRpm'):
                temp_dict.update(dict([values]))
        elif 'KP' in token:
            values = [x.strip() for x in token.split(':')]
            if (len(values) == 2) and (values[0] == 'KP'):
                temp_dict.update(dict([values]))
        elif 'Error' in token:
            values = [x.strip() for x in token.split(':')]
            if (len(values) == 2) and (values[0] == 'Error'):
                temp_dict.update(dict([values]))
        elif 'MotorOutRpm' in token:
            values = [x.strip() for x in token.split(':')]
            if (len(values) == 2) and (values[0] == 'MotorOutRpm'):
                temp_dict.update(dict([values]))
        
    df = df.append(temp_dict, ignore_index=True)
    return df

In [12]:
# Setup empty Dataframe
df =pd.DataFrame()

# setup a timeout in seconds and grab the start time
timeout = 10 #[secs]
timeout_start = time.time()

# capture until the timeout is reached
while time.time() < timeout_start + timeout:
    df=update(df)
    
# Clean-up the NaN values
df.KP  = df.KP.astype(float)
df.Error = df.Error.astype(float)
df.ActualRpm  = df.ActualRpm.astype(float)
df.DesiredRpm = df.DesiredRpm.astype(float)
df.MotorOutRpm = df.MotorOutRpm.astype(float)

In [ ]:
df

In [13]:
# Plot the Desired, Actual, and Output RPM values
p = figure(plot_width=800, plot_height=400, title='Motor Control Plot')
p.line(df.index, df.ActualRpm, legend="Actual Rpm", line_color="blue")
p.line(df.index,df.DesiredRpm, legend="Desired Rpm", line_color="red")
p.line(df.index,df.MotorOutRpm,legend=" Motor Out", line_color="purple")
p.xaxis.axis_label = "Time Elapsed(s)"
p.yaxis.axis_label = "RPM"
p.legend.location = "bottom_left"
p.legend.border_line_width = 2
p.legend.border_line_color = "black"
p.legend.border_line_alpha = 0.3
show(p)

In [8]:
p2 = figure(plot_width=800, plot_height=400, title='Error & KP')
p2.line(df.index,df.KP, legend="KP",line_color="green")
p2.line(df.index,df.Error, legend="Error", line_color="red")
p2.xaxis.axis_label = "Time Elapsed(s)"
p2.yaxis.axis_label = "Value"
p2.legend.location = "bottom_left"
p2.legend.border_line_width = 2
p2.legend.border_line_color = "black"
p2.legend.border_line_alpha = 0.3
show(p2)

In [ ]:
pd.to_csv("motorcontrolData.csv")